# Case 2 - Question 3

In this notebook, we will attempt to create a 60/40 Canadian Equity-Bond Mix as a benchmark. We will compare this portfolio with the returns reported from CPPIB and of the reference portfolio.

## Canadian Equity Bond Mix
- Gathering the data to create this was significantly more difficult than expected.
- For the **equity** portion, we chose to use the TSX60 (large-cap Canadian Firms) index as our representative index
    - Dividends are immediately reinvested on dividend date (at close)
- For the **bond** portion, we will use BlockRock's iShares Core Canadian Universe Bond Index ETF (XBB.TO) to track domestic bond returns
    - Similarly, dividends are reinvested on dividend date at close
 
## Timeframe
- The case notes that CPPIB uses fiscal year format. That is,
    - CPPIB reports its annual performance according to its fiscal calendar (April - March) we will take annual earnings using this time range
    - For example: 2000 returns will be from the time frame April 1999 - March 2000
    - This is supported by the return characteristics around the financial crisis

In [1]:
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
tsx60 = pd.read_sas("tsx60.sas7bdat", encoding = 'ISO-8859-1')

In [3]:
tsx60.head()

,GVKEYX,DATADATE,CONM,TIC,DVPSXM,PRCCM
0,118341,1999-03-31,S&P/TSX 60 Index,CI0029,NaN,384.58
1,118341,1999-04-30,S&P/TSX 60 Index,CI0029,NaN,410.85
2,118341,1999-05-31,S&P/TSX 60 Index,CI0029,NaN,400.07
3,118341,1999-06-30,S&P/TSX 60 Index,CI0029,NaN,411.04
4,118341,1999-07-31,S&P/TSX 60 Index,CI0029,NaN,415.10


In [4]:
portfolio_rets = pd.DataFrame(columns = ["monthid", "equity_ret"])

In [5]:
equity_portfolio = tsx60[["DATADATE", "DVPSXM", "PRCCM"]].copy()

In [6]:
equity_portfolio.columns = ["DATE", "DIVIDEND", "PRC"]
equity_portfolio = equity_portfolio.fillna(0)

In [7]:
num_shares = 1
for row in equity_portfolio.iterrows():
    data = row[1]
    
    dividend = data["DIVIDEND"] * num_shares
    
    shares_repurchased = dividend / data.PRC # Div / Prc = shares repurchased
    num_shares += shares_repurchased
    
    equity_portfolio.loc[row[0], "shares_holding"] = num_shares

In [8]:
counter = 0
for i in range(12, len(equity_portfolio), 12):
    portfolio_rets.loc[counter, "monthid"] = i
    portfolio_rets.loc[counter, "equity_ret"] = (equity_portfolio.loc[i, "shares_holding"] * equity_portfolio.loc[i, "PRC"]) / (equity_portfolio.loc[i - 12, "shares_holding"] * equity_portfolio.loc[i - 12, "PRC"]) - 1
    counter += 1
portfolio_rets


,monthid,equity_ret
0,12,0.47925
1,24,-0.22122
2,36,0.008351
3,48,-0.175403
4,60,0.342734
5,72,0.144315
6,84,0.301257
7,96,0.123492
8,108,0.063716
9,120,-0.304391


***
## Read Bonds ETF
For the bond portion, we will use BlockRock's iShares Core Canadian Universe Bond Index ETF (XBB.TO) to track domestic bond returns
- Similarly, dividends are reinvested on dividend date at close

Note that since the ETF was only created on Nov 23 2001, we will use $r_f$ values from WRDS for the starting dates as a proxy for bond returns

In [9]:
xbb = pd.read_csv("XBB.TO.csv")
xbb_dividends = pd.read_csv("XBB.TO-div.csv")

In [10]:
xbb.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2001-11-23,26.650000,26.750000,26.600000,26.750000,12.415666,6600.0
1,2001-11-26,26.799999,26.799999,26.719999,26.719999,12.401739,21300.0
2,2001-11-27,26.700001,26.750000,26.650000,26.750000,12.415666,46300.0
3,2001-11-28,26.950001,26.950001,26.900000,26.900000,12.485283,800.0
4,2001-11-29,27.000000,27.100000,26.950001,27.049999,12.554912,23200.0


In [11]:
xbb_dividends.head()

,Date,Dividends
0,2001-12-24,0.801
1,2002-06-24,0.679
2,2002-12-23,0.657
3,2003-06-23,0.663
4,2003-12-29,0.677


In [12]:
bonds = pd.merge(xbb, xbb_dividends, on="Date", how="left")

In [13]:
bonds["Dividends"].fillna(0, inplace=True)
bonds.dropna(inplace=True)

In [14]:
bond_portfolio = bonds[["Date", "Close", "Dividends"]]

In [15]:
bond_portfolio

,Date,Close,Dividends
0,2001-11-23,26.750000,0.000
1,2001-11-26,26.719999,0.000
2,2001-11-27,26.750000,0.000
3,2001-11-28,26.900000,0.000
4,2001-11-29,27.049999,0.000
...,...,...,...
2375,2011-03-24,29.709999,0.000
2376,2011-03-25,29.600000,0.089
2377,2011-03-28,29.570000,0.000
2378,2011-03-29,29.500000,0.000


In [16]:
# Reinvest dividends
num_shares = 1
for row in bond_portfolio.iterrows():
    data = row[1]
    
    dividend = data["Dividends"] * num_shares
    
    shares_repurchased = dividend / data.Close # Div / Prc = shares repurchased
    num_shares += shares_repurchased
    
    bond_portfolio.loc[row[0], "shares_holding"] = num_shares
bond_portfolio = bond_portfolio[bond_portfolio["Date"] >= "2002-03-31"]

In [17]:
bond_portfolio.reset_index(inplace=True)
bond_portfolio.head()

,index,Date,Close,Dividends,shares_holding
0,91,2002-04-01,25.799999,0.0,1.030514
1,92,2002-04-02,25.850000,0.0,1.030514
2,93,2002-04-03,25.799999,0.0,1.030514
3,94,2002-04-04,26.000000,0.0,1.030514
4,95,2002-04-05,26.049999,0.0,1.030514


In [18]:
# Convert to monthly frequency
bond_portfolio_monthly = pd.DataFrame(columns = ["Date", "Close", "shares_holding","portfolio value"])
seen_year_months = set()

for row in bond_portfolio.iterrows():
    data = row[1]

    year_month = data["Date"][:7]
    if year_month in seen_year_months:
        continue

    bond_portfolio_monthly.loc[row[0], ["Date", "Close", "shares_holding","portfolio value"]] = [data["Date"], data["Close"], data["shares_holding"], data["shares_holding"] * data["Close"]] 
    seen_year_months.add(year_month)

bond_portfolio_monthly.loc[row[0], ["Date", "Close", "shares_holding","portfolio value"]] = [data["Date"], data["Close"], data["shares_holding"], data["shares_holding"] * data["Close"]] 

    

In [19]:
bond_portfolio_monthly.reset_index(inplace=True)
# Add last value
last_val = bond_portfolio.iloc[-1]
bond_portfolio_monthly.loc[len(bond_portfolio_monthly), ["Date", "Close", "shares_holding","portfolio value"]] = [last_val["Date"], last_val["Close"], last_val["shares_holding"], last_val["shares_holding"] * last_val["Close"]]

bond_portfolio_monthly

,index,Date,Close,shares_holding,portfolio value
0,0.0,2002-04-01,25.799999,1.030514,26.587268
1,22.0,2002-05-01,26.35,1.030514,27.154051
2,44.0,2002-06-03,26.5,1.030514,27.308629
3,64.0,2002-07-02,26.35,1.05712,27.855101
4,86.0,2002-08-01,26.85,1.05712,28.383661
...,...,...,...,...,...
105,2203.0,2011-01-04,29.74,1.508922,44.875347
106,2223.0,2011-02-01,29.530001,1.513894,44.705288
107,2242.0,2011-03-01,29.51,1.518935,44.823773
108,2263.0,2011-03-30,29.48,1.523502,44.912842


In [20]:
## We'll use r_f as a proxy for our "bond" returns for years 2000-2002

In [21]:
rf = pd.read_sas("rf.sas7bdat", encoding = 'ISO-8859-1')

In [22]:
rf_returns = rf[rf["DATEFF"] <= "2002-03-31" ]

In [23]:
rf_returns.tail()

,DATEFF,RF
31,2001-11-30,0.0017
32,2001-12-31,0.0015
33,2002-01-31,0.0014
34,2002-02-28,0.0013
35,2002-03-28,0.0013


In [24]:
# Add to portfolio_rets

portfolio_rets.head()

,monthid,equity_ret
0,12,0.47925
1,24,-0.22122
2,36,0.008351
3,48,-0.175403
4,60,0.342734


In [25]:
# Add rf bond yields
counter = 0
for i in range(0, len(rf_returns), 12):
    if i + 12 == len(rf_returns):
        portfolio_rets.loc[counter, "bond_ret"] = rf_returns.loc[i:, "RF"].sum()
    else:
        portfolio_rets.loc[counter, "bond_ret"] = rf_returns.loc[i:i+12, "RF"].sum()
    counter += 1
portfolio_rets

,monthid,equity_ret,bond_ret
0,12,0.47925,0.0523
1,24,-0.22122,0.0615
2,36,0.008351,0.0282
3,48,-0.175403,NaN
4,60,0.342734,NaN
5,72,0.144315,NaN
6,84,0.301257,NaN
7,96,0.123492,NaN
8,108,0.063716,NaN
9,120,-0.304391,NaN


In [26]:
# Add bond yields from XBB
counter = 3
for i in range(12, len(bond_portfolio_monthly), 12):
    portfolio_rets.loc[counter, "bond_ret"] = (bond_portfolio_monthly.loc[i, "portfolio value"] / bond_portfolio_monthly.loc[i - 12, "portfolio value"]) - 1
    counter += 1
portfolio_rets


,monthid,equity_ret,bond_ret
0,12,0.47925,0.052300
1,24,-0.22122,0.061500
2,36,0.008351,0.028200
3,48,-0.175403,0.103485
4,60,0.342734,0.114853
5,72,0.144315,0.033344
6,84,0.301257,0.040358
7,96,0.123492,0.052970
8,108,0.063716,0.045682
9,120,-0.304391,0.055103


In [27]:
# Now we can calculate the 60/40 portfolio return
portfolio_rets["60/40 Benchmark"] = portfolio_rets["equity_ret"] * 0.6 + portfolio_rets["bond_ret"] * 0.4

In [28]:
portfolio_rets["monthid"] /= 12
portfolio_rets["monthid"] += 1999
portfolio_rets.columns = ['Year', 'equity_ret', 'bond_ret', '60/40 Benchmark']

In [29]:
portfolio_rets.astype(float).round(4)

,Year,equity_ret,bond_ret,60/40 Benchmark
0,2000.0,0.4793,0.0523,0.3085
1,2001.0,-0.2212,0.0615,-0.1081
2,2002.0,0.0084,0.0282,0.0163
3,2003.0,-0.1754,0.1035,-0.0638
4,2004.0,0.3427,0.1149,0.2516
5,2005.0,0.1443,0.0333,0.0999
6,2006.0,0.3013,0.0404,0.1969
7,2007.0,0.1235,0.0530,0.0953
8,2008.0,0.0637,0.0457,0.0565
9,2009.0,-0.3044,0.0551,-0.1606
